In [389]:
import pandas as pd
import numpy as np
import numpy.ma as ma
import time
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score

There are 6040(max user id) users and 3952(max movie id) Movies in our database

Strategy
<ol>
    <li>Create a Utility Matrix(M) with the size of Users and Movies</li>
    <li>Update M with the ratings from training dataset</li>
    <li>Decide for d dimension (for U & V)</li>
    <li>Implement the decomposition algorithm</li>
    <li>Initialize U & V matrices</li>
    <li>Train and find optimum U & V matrices</li>
    <li>Evaluate on test set using RMSE values</li>
</ol>

<img src="./images/IMG_1387.jpeg" style="width: 75%; height: auto;">

In [27]:
# Reading the data
user_col_names = ["UserID", "Gender", "Age", "Occupation", "Zipcode"]
udf = pd.read_csv("ml-1m/users.dat", sep="::", header=None, names=user_col_names, engine="python")

movies_col_names = ["MovieID", "Title", "Genres"]
mdf = pd.read_csv("ml-1m/movies.dat", sep="::", header=None, names=movies_col_names, engine="python")

ratings_col_names = ["UserID", "MovieID", "Rating", "Timestamp"]
rdf = pd.read_csv("ml-1m/ratings.dat", sep="::", header=None, names=ratings_col_names, engine="python")

In [28]:
print(f"UserID: min = {udf.UserID.min()}, max = {udf.UserID.max()}")
print(f"MovieID: min = {mdf.MovieID.min()}, max = {mdf.MovieID.max()}")

UserID: min = 1, max = 6040
MovieID: min = 1, max = 3952


In [22]:
# Dividing the data into 5 folds
RandomState = 42

kf = KFold(n_splits=5, shuffle=True, random_state=RandomState)
Folds = []
for train_index, test_index in kf.split(rdf):
    Folds.append((rdf.iloc[train_index, :], rdf.iloc[test_index, :]))

In [369]:
def update_M(m, df):
    for index, row in df.iterrows():
        m[row["UserID"]-1, row["MovieID"]-1] = row["Rating"]
    return m

def normalize_M(M):
    m = M.copy()
    m2 = ma.masked_array(m, mask=m==0)
    users_mean = m2.mean(axis=1)
    ma.set_fill_value(users_mean, 0)
    movies_mean = m2.mean(axis=0)
    ma.set_fill_value(movies_mean, 0)
    
    for i in range(m.shape[0]):
        for j in range(m.shape[1]):
            if m[i, j] != 0:
                m[i,j] = m[i,j] - (users_mean[i] + movies_mean[j])/2 
    return m, users_mean.filled(), movies_mean.filled()

# RMSE calculation
def calc_rmse(M, U, V, root=True):
    masked = ma.masked_array(M, mask=M==0)
    P = np.dot(U, V)     
    s = np.sum((masked-P)**2)
    if not root:
        return s
    return np.sqrt(s/masked.count())

def decompose_U(M, U, V):
    n, d = U.shape
    for r in range(n):
        for s in range(d):
            U[r,s]= Urs(M, U, V, r, s)
    return U

def Urs(M, U, V, r, s):
    M_slice = ma.masked_array(M, mask=M==0)[r, :]
    V_slice = ma.masked_array(V[s, :], M_slice.mask)
    sum_array=np.matmul(U[r,:],V[:])-(U[r,s]*V[s,:])
    numerator = np.sum(V[s,:]*(M_slice-sum_array))
    denominator = np.sum(np.square(V_slice))
    if denominator == 0:
        return 0
    return numerator/denominator


def decompose_V(M, U, V):
    d, m = V.shape
    for s in range(m):
        for r in range(d):
            V[r,s]= Vrs(M, U, V, r, s)
    return V

def Vrs(M, U, V, r, s):
    M_slice = ma.masked_array(M, mask=M==0)[:, s]
    U_slice = ma.masked_array(U[:, r], M_slice.mask)
    sum_array=np.matmul(U[:],V[:, s])-(V[r,s]*U[:, r])
    numerator = np.sum(U[:, r]*(M_slice-sum_array))
    denominator = np.sum(np.square(U_slice))
    if denominator == 0:
        return 0
    return numerator/denominator

In [ ]:
d = 3 # TODO: Decide this later 
n =udf.UserID.max()
m = mdf.MovieID.max()
threshold = 999 # TODO: Decide this later
k = 1 # TODO: Decide this later
results = []

for i, (train, test) in enumerate(Folds):
    
    t0 = time.time()
    # Step 1 : Create Utility Matrix (M)
    M = np.zeros((n, m))
    print("Utility Matrix is created, now being updated.")
    # Step 2: Fill Utility Matrix with Ratings from train set 
    M = update_M(M, train)
    # Step 3: Normalize Utility Matrix [Preprocessing]
    M_norm, users_mean, movies_mean = normalize_M(M)
    t1 = time.time()
    print(f"Utility Matrix is ready, took {round(t1 - t0, 2)} sec")
    
    
    # Step 4: Initialize U,V matrices with 0 because we normalized M
    U = np.zeros((n, d))
    V = np.zeros((d, m))
    
    # Step 5: Performing the Optimization
    optim_rmse_history = []
    optim_rmse = 9999
    print("Optimization has started.")
    t2 = time.time()
    while (optim_rmse >= threshold) and (k > 0):
        t4 = time.time()
        U = decompose_U(M, U, V)
        t5 = time.time()
        print(f"Decomposition of U took {round(t5-t4, 2)} sec")
        
        V = decompose_V(M, U, V)
        t6 = time.time()
        print(f"Decomposition of V took {round(t6-t5, 2)} sec")
        
        optim_rmse = calc_rmse(M, U, V)
        optim_rmse_history.append(optim_rmse)
        k = k - 1
    t3 = time.time()
    print(f"Optimization is finished in {round(t3 - t2, 2)} sec" )
    
    P = np.dot(U, V)
    users = test["UserID"] - 1
    movies = test["MovieID"] - 1
    y_pred = P[users, movies]
    
    # Undo normalization
    y_pred = y_pred + (users_mean[users] + movies_mean[movies])/2
    
    y_test = test["Rating"].values
    
    test_rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    # The mean squared error
    print('Root mean squared error: %.2f' % test_rmse)
    
    r2 = r2_score(y_test, y_pred)
    # The coefficient of determination: 1 is perfect prediction
    print('Coefficient of determination: %.2f'% r2_score(y_test, y_pred))
    
    res = {
        "fold": i,
        "M_preprocessing": round(t1-t0, 2),
        "Decomposition_U": round(t5-t4, 2),
        "Decomposition_V": round(t6-t5, 2),
        "Optimization": round(t3 - t2, 2),
        "U": U,
        "V": V,
        "optim_rmse_history": optim_rmse_history,
        "test_rmse": test_rmse,
        "r2_score":r2
    }
    
    results.append(res)
    break